<a href="https://colab.research.google.com/github/EdwardGerman/Columnar-Transposition-Cipher/blob/main/Letters_pairs_preproc_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preproces for **letters pairs**

start processing!!!

##About
There is a task of decoding "crypted" text, which is based on the number of occurrences of pairs of letters in "natural language text".

This notebook is designed to pre-process the text to meet the requirements of the coding task, namely:
- the text contains only small letters and a space;
- all words write successively on one line - there are no paragraphs (\n) and other hidden characters.

Additional text processing is possible, which depends directly on the properties of a specific type of text.

---
Є задача декодування "зашифрованого" тексту, яка базується на кількості частоти входжень пар літер в "natural language text".

Цей ноутбук призначений для попередньої обробки тексту, щоб відповідати умовам задачі кодування, а саме:
- текст містить тільки маленьки літери та пробіл;
- всі слова їдуть підряд в один рядок -  відсутні абзаци (\n) та інші сховані символи.

Можлива додаткова обробка тексту, яка залежить беспосередньо від особливостей конкретного типу тексту.



The following texts were used for train:
- (1 HS) "Hamlet" by Shakespeare ([here](https://shakespearestudyguide.com/Hamlet%20Text.html)) [Without expressions like "BERNARDO:"]
- (2 TB) Torah, Sefer Bereshit in English ([here](https://www.tanach.us/Pages/About.html))
- (3 AR) Asimov "I, Robot"  ([here](https://royallib.com/book/Asimov_Isaac/I_Robot.html))
- (4 HP) "Harry Potter and the Philosopher's Stone" ([here](https://github.com/amephraim/nlp/blob/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt))
- (5 GG) Genetics : From Genes to Genomes. 6th edition ([here](http://skgjx.whu.edu.cn/Public/upfile/article/202103031656469260.pdf)) [Intro & parts 1,2,4. My edit from PDF]
- (6 SA) Scientific articles: from IEEE, [medium.com](https://medium.com/) etc.
- (7 NA) News articles: [New York Times](https://www.nytimes.com/international/), [Fox News](https://www.foxnews.com/), [BBC](https://www.bbc.com/news) etc.
- (8 CC) Coments in chats








In "Hamlet" have been individually removed words like "BERNARDO:" . Now all text files are processed the same, using a special function.

В "Hamlet" окремо видалили слова на кшталт "BERNARDO:"
Тепер всі текстові файли овробляються однаково, за допомогою спеціальної функції.



---
---
## Import & mount

In [4]:
import os
import re
import copy
from time import time
import numpy as np
import pandas as pd

# from matplotlib import pyplot as plt
# plt.rcParams['figure.figsize'] = [15, 6]

In [5]:
# Mount GitHub
!git clone https://github.com/EdwardGerman/Columnar-Transposition-Cipher.git  # clone repository
%ls  # checking whether all files are present
drch = '/content/Columnar-Transposition-Cipher' # Path to data

folder_r = 'Data'
folder_w = 'Data_pp'

fatal: destination path 'Columnar-Transposition-Cipher' already exists and is not an empty directory.
Columnar-Transposition-Cipher/  sample_data/


---
---
## Funcfions

#### Space_n_letters()
Function for count of all letters and spaces in preproc text

In [6]:
def Space_n_Letters(pp_text, p = False):
    chars_after_PP = len(pp_text)
    spaces = pp_text.count(' ')
    letters = chars_after_PP - spaces

    if p == False:
        return [chars_after_PP, letters, spaces]
    else:
        print('Symbols number:',chars_after_PP)
        print('Spaces number:',spaces)
        print('Only letters: ', letters,'\n')
    # print('Only letters (2): ', len(list(filter(str.isalpha, pp_text))),'\n')

#### CharacterCounts()
Counts the number of each character in the text and print sorted [by decreasing] list (dictionary).

!Noties! optiotal:
* Upper letter = lower letter
* Multyspace -> space

In [7]:
# Returns sorted dictionary {letter : count} from largest to smallest number
def CharactersCount(orig_text, multispace = False, uppercase = False):
    text = copy.copy(orig_text)
    if multispace == False: text = re.sub(r' +', ' ', text)
    if uppercase  == False: text = text.lower()

    counts = {}   # Creating empty dictionary
    for char in text:    # Loop for count characters
        if char in counts: counts[char] += 1
        else:              counts[char]  = 1
    return dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

---
---
---
## Processing
Order is following:
- to lower letters;
- remove [apostrophe](https://en.wikipedia.org/wiki/Apostrophe) in the middle of the words;
- keep only letters and spaces (all rest convert to space);
- multispace to space;





In [8]:
def TextPreprocessing(orig_txt):
    pp_text = copy.copy(orig_txt)

    pp_text = pp_text.lower()                                   # to lower letters
    pp_text = re.sub(r"(?<=[a-z])[`'’](?=[a-z])", '', pp_text)  # remove `'’

    pp_text = re.sub(r'[^a-z ]', ' ', pp_text)  # only letters and spaces (all rest convert to space)
    pp_text = re.sub(r' +', ' ', pp_text)       # multyspace to space

    if pp_text.endswith(' '): pp_text = pp_text[:-1]

    return pp_text

---

Get list of all files in directory

In [9]:
# Get list of all files in directory
file_list = os.listdir(os.path.join(drch, 'Data'))
# file_list

Creating DataFrames for statistics

In [10]:
chars_count_df = pd.DataFrame()                     # Df for count of all chars in texts
chars_count_pp_df = pd.DataFrame()                  # Df for count of chars in texts afrer preproc
                                                    # (only letters and spaces)
chars_number_df = pd.DataFrame(index=file_list,     # DF what contain chars number before & after PP
                               columns=['Chars before PP', 'Chars after PP', 'Letters', 'Spaces'])

Preprocessing in the loop:
- read text from file;
- count of chars in original text;
- exactly `Preprocessing`;
- count of chars in text preprocessing;
- write preprocessing text to file;



In [11]:
all_pp_text = ''

# Loop for each file:
for text_file in file_list:
    file_name = text_file.replace('.txt', '')   # Only file name, without extension

    # Read:
    path_r = os.path.join(drch, folder_r, text_file)
    with open(path_r, 'r', errors='ignore') as file:
        orig_text = file.read()

    # Count of characters in original text (before pp):
    ser = pd.DataFrame([CharactersCount(orig_text)]).T
    ser.columns = [file_name]
    chars_count_df = pd.concat([chars_count_df, ser], axis=1)

    # Preprocessing process:
    pp_text = TextPreprocessing(orig_text)

    # Count of characters in preproc text:
    ser = pd.DataFrame([CharactersCount(pp_text)]).T
    ser.columns = [file_name]
    chars_count_pp_df = pd.concat([chars_count_pp_df, ser], axis=1)

    # Write:
    path_w = os.path.join(drch, folder_w, file_name + '_pp.txt')
    with open(path_w, 'w') as file:
        file.write(pp_text)

    # Count:
    chars_before_PP = len(orig_text)
    chars_number_df.loc[text_file] = [chars_before_PP] + Space_n_Letters(pp_text)

    # Add preproc text to "All text" variable
    all_pp_text = all_pp_text + ' ' + pp_text

Finishing of formation statistic DataFrames

In [12]:
# --------------------------------------------------------------------------------------------------
# Finishing for "chars_count_df"
chars_count_df = chars_count_df.fillna(0)           # Change 'Nan' to '0'
chars_count_df = chars_count_df.astype(int)         # All values is integer

chars_count_df['Sum'] = chars_count_df.sum(axis=1)  # New column with sums for each chars in all files
chars_count_df = chars_count_df.sort_values(by='Sum', ascending=False)      # sort by count

# --------------------------------------------------------------------------------------------------
# Finishing for "chars_count_pp_df"
chars_count_pp_df = chars_count_pp_df.fillna(0)     # Change 'Nan' to '0'
chars_count_pp_df = chars_count_pp_df.astype(int)   # All values is integer

chars_count_pp_df['Sum'] = chars_count_pp_df.sum(axis=1)# New column with sums for each chars in all files
chars_count_pp_df = chars_count_pp_df.sort_values(by='Sum', ascending=False)# sort by count

# --------------------------------------------------------------------------------------------------
# Finishing for "chars_number_df"
chars_number_df.loc['All texts'] = chars_number_df.sum()

---

Write to text file all preprocessing texts

In [13]:
all_pp_text = all_pp_text[1:]
path_w = os.path.join(drch, folder_w, 'All texts' + '_pp.txt')
with open(path_w, 'w') as file:
    file.write(all_pp_text)

## Statistics

#### 1. Characters number
- All chars number before preproc
- All chars number after preproc
- Letters number after preproc
- Spaces number after preproc

In [14]:
chars_number_df

,Chars before PP,Chars after PP,Letters,Spaces
Asimov_I_Robot.txt,400697,377897,307919,69978
Harry_Potter_I.txt,439742,412898,335007,77891
Sci_articles.txt,360592,325498,268596,56902
News_articles.txt,101735,98258,81777,16481
Hamlet_BB.txt,242587,211013,170156,40857
Genetics_124.txt,396178,379158,316667,62491
Chat_Comments.txt,288735,277918,227565,50353
Torah_Bereshit.txt,199164,181245,144894,36351
All texts,2429430,2263885,1852581,411304


#### 2. Number of each character in each file `before preproc`

In [15]:
pd.set_option('display.max_rows', None) # Print all rows (features) in DF
display(chars_count_df)
pd.reset_option('display.max_rows')     # Default setting: print print only first & final 5 rows

,Asimov_I_Robot,Harry_Potter_I,Sci_articles,News_articles,Hamlet_BB,Genetics_124,Chat_Comments,Torah_Bereshit,Sum
,66953,70803,53178,16153,41949,56035,47892,37328,390291
e,37488,39915,32942,10076,20205,41548,27325,18716,228215
t,29480,29048,25495,6955,15720,26468,21181,12030,166377
a,23923,26590,21330,6961,12924,23792,18257,13194,146971
o,24728,26141,19496,6038,14297,25416,18233,11078,145427
i,21820,20815,20546,6265,11417,23969,16753,8831,130416
n,21648,21825,19138,5936,11086,22603,15676,8783,126695
s,18756,19714,18743,5551,11057,22440,15257,8770,120288
r,17367,21650,17301,5361,10063,19365,13218,7964,112289
h,17535,22531,10253,3599,10512,14595,10999,12078,102102


#### 3. Number of each character in each file `after preproc`

In [16]:
chars_count_pp_df['Sum'] = chars_count_pp_df.sum(axis=1)
chars_count_pp_df = chars_count_pp_df.sort_values(by='Sum', ascending=False)

chars_count_pp_df

,Asimov_I_Robot,Harry_Potter_I,Sci_articles,News_articles,Hamlet_BB,Genetics_124,Chat_Comments,Torah_Bereshit,Sum
,69978,77891,56902,16481,40857,62491,50353,36351,822608
e,37488,39915,32942,10076,20205,41548,27325,18716,456430
t,29480,29048,25495,6955,15720,26468,21181,12030,332754
a,23923,26590,21330,6961,12924,23792,18257,13194,293942
o,24728,26141,19496,6038,14297,25416,18233,11078,290854
i,21820,20815,20546,6265,11417,23969,16753,8831,260832
n,21648,21825,19138,5936,11086,22603,15676,8783,253390
s,18756,19714,18743,5551,11057,22440,15257,8770,240576
r,17367,21650,17301,5361,10063,19365,13218,7964,224578
h,17535,22531,10253,3599,10512,14595,10999,12078,204204


In [17]:
lst = sorted(chars_count_pp_df.index.tolist())
print(lst)
# print(chars_count_pp_df.index.tolist().sort())

[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


---

---
---
##Hamlet

### Bli "BERNARDO:"

In [18]:
# drch = '/content/drive/MyDrive/DS small/Letters pairs/'

# path_r = os.path.join(drch, 'Data_add', 'Hamlet.txt')
# with open(path_r, 'r') as file:
#     orig_Hamlet = file.read()

# Hamlet_BB = re.sub(r'\b[A-Z]+\b:', '', orig_Hamlet)   # Regular expression for "BERNARDO:" etc.

# path_w = os.path.join(drch, 'Data_add', 'Hamlet_BB.txt')
# with open(path_w, 'w') as file:
#     file.write(Hamlet_BB)


---
---
---
#!!!
Draft version (this code don`t use in this project)

In [19]:
# text   = "When I see the title, I was scared that it would required a NSFW tag, lol! It's safe to assume that's why we didn't get to see the basement of the Inn Seeing all those rabbits tells me this is moon guard."
# text_1 = "When I see the tit3le, I was sca4red that it w5ould required a 1NSFW tag, lo6l! It's safe to as9sume that'0s why w2e didn't get to see the bas34ement of the Inn See5ing all6 7those r0a0b8b7its tel6ls me th7is is moon gua8rd9."
# text_2 = "When I see the title, I was scared that it would required a NSFW tag, lol! It's safe to assume that's why we didn't get to see the basement of the Inn Seeing all those rabbits tells me this is moon guard."


In [20]:
# text = "Це рядок 123 з кількома 4 цифрами в словах: b2a, a3bc, 1def, ghi4, eff56r, fgf34."
# # Використовуємо регулярний вираз для видалення цифр, які оточені буквами
# modified_text = re.sub(r'(?<=[a-zA-Z])\d(?=[a-zA-Z])', '', text)
# print(modified_text)

In [21]:
# text = "Це рядок: b`a, ab~c, d'e!f, ghi~, ef'fr, fgf'."

# modified_text = re.sub(r"(?<=[a-zA-Z])[`~!'’](?=[a-zA-Z])", '', text)
# print(modified_text)

In [22]:
# file_name = 'Torah_Bereshit_Bereshit.txt'
# folder = 'Data_add/'
# path = os.path.join(drch, folder, file_name)

# with open(path, 'r') as file:
#     orig_text = file.read()

# pp_text = copy.copy(orig_text)
# print(orig_text.count("’"))
# print(orig_text.count("'"))
# print(orig_text.count("‘"),'\n')


# pp_text = pp_text.lower()                                   # to lower letters
# pp_text = re.sub(r"(?<=[a-z])[`'’](?=[a-z])", '', pp_text)  # remove `'’
# print(pp_text)

# print(pp_text.count("’"))
# print(pp_text.count("'"))
# print(pp_text.count("‘"),'\n')

# file_name = 'Torah_Bereshit_Bereshit_pp.txt'
# folder = 'Data_add/'
# path = os.path.join(drch, folder, file_name)

# with open(path, 'w') as file:
#     file.write(pp_text)